In [ ]:
#     In this project, historical data for various stocks is pulled from the Polygon API.  Using an CNN-LSTM neural network
# model, as well as other forms of analysis, the stock data is scrutinized and used to train an algorithm that predicts
# the future stock price 1 day ahead.
#
#     This is the first piece of code for the stock project, responsible for API calls.  The code makes a call, formats 
# the request into a pandas dataframe (df), and imports the df to both an Amazon Web Services (AWS) database, and a csv
# in the local path.  Additionally, as new data is pulled, this program updates the database with the new data.

In [ ]:
# import libraries

import requests
import pandas as pd
import time
import psycopg2 as ps
import os
import re
from datetime import datetime, timedelta

In [ ]:
# construct list of API queries in example format-> https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/
# 2021-07-22/2022-07-26?adjusted=true&sort=asc&limit=120&apiKey=yNwYdJmuOjYxtPaHusxz_0_pekcZ2ENi

def const_api_urls(ticker_list, API_KEY):
    # grabs today's date, as well as the date of the same business day from last year for the time window
    
    now = datetime.now() 
    end_date = now.strftime("%Y-%m-%d")
    start_date = (datetime.strptime(end_date, "%Y-%m-%d") - timedelta(days=728)).strftime("%Y-%m-%d") 
    adjusted = 'true'
    multiplier = '1'
    resolution = 'day'
    sort = 'asc'
    limit = '1000'
    api_urls = []
    for ticker in ticker_list:
        api_urls.append("https://api.polygon.io/v2/aggs/ticker/%s/range/%s/%s/%s/%s?adjusted=%s&sort=%s&limit=%s&apiKey=%s" 
              % (ticker, multiplier, resolution, start_date, end_date, adjusted, sort, limit, API_KEY))
    return api_urls

In [ ]:
# check last date in database
def check_date(cursor, tbl_name):
    query = (""" SELECT MAX(Date) FROM %s """ % (tbl_name))
    cursor.execute(query)
    return cursor.fetchone()

In [ ]:
# end of defined functions

In [ ]:
# perform API requests (note: a max of 5 API requests are allowed per minute)

#ticker_list = [
    #'JNJ', 'PG', 'CVX', 'BAC', 'VZ', 'UPS', 'C',
    #'USB', 'AON', 'OXY', 'MCO', 'MCK', 'HPQ', 'KR',
    #'AAPL', 'CE', 'ALLY', 'DVA', 'STOR', 'FND'          
#]
#ticker_list = [
    #'PDBC', 'QQQ', 'SPY', 'GDX', 'VOO'
#]
ticker_list = [
    'XLB', 'XLC', 'XLE', 'XLI', 'XLF', 'XLK'
]
API_KEY = 'yNwYdJmuOjYxtPaHusxz_0_pekcZ2ENi'
api_urls = const_api_urls(ticker_list, API_KEY)
remove_keys = ['vw','t','n']
df = pd.DataFrame()

for count, url in enumerate(api_urls):
    if (count+1) % 5 == 0:
        time.sleep(60) # wait 1 minute every 5 requests, to not exceed API request limit
        
    response = requests.get(url).json()['results']
    if (count) == 0:
        unix_timestamps = [{k : v for k, v in day.items() if k == 't'} for day in response]
        for day in unix_timestamps:
            day.update((k, v/1000) for k, v in day.items())
            
        
        df["Date"] = [day['t'] for day in unix_timestamps]
        
    indiv_data = [{k : v for k, v in day.items() if k not in remove_keys} for day in response]
    df["Open_%s" % (ticker_list[count])] = [day['o'] for day in indiv_data]
    df["Close_%s" % (ticker_list[count])] = [day['c'] for day in indiv_data]
    df["High_%s" % (ticker_list[count])] = [day['h'] for day in indiv_data]
    df["Low_%s" % (ticker_list[count])] = [day['l'] for day in indiv_data]
    df["Volume_%s" % (ticker_list[count])] = [day['v'] for day in indiv_data]
    
    
    response.clear()
    indiv_data.clear()


In [ ]:
df.head()

In [ ]:
 df.info()

In [ ]:
# change unix time stamp column to datetime

df['Date'] = [datetime.fromtimestamp(d) for d in df['Date']]


In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
# change df data types to SQL datatypes
replacement_dtypes = {'object':'varchar','float64': 'float', 'int64':'int','datetime64[ns]': 'timestamp'}
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacement_dtypes)))

In [ ]:
host = 'db-1.crs7hg7ymswn.us-east-2.rds.amazonaws.com'
name = 'BerkHathPort'
user = 'saajanrpatel'
password = 'Tampa!bay21'
conn_str = "host=%s dbname=%s user=%s password=%s" % (host, name, user, password)
print(conn_str)

In [ ]:
# open connection to database

tbl_name = 'XL_portfolio'
csv_name = tbl_name + '.csv'
conn = ps.connect(conn_str)
cursor = conn.cursor()
cursor.execute("drop table if exists %s;" % (tbl_name))
cursor.execute("create table %s (%s)" % (tbl_name, col_str))
df.to_csv(csv_name, header = df.columns, index = False, encoding = 'utf-8')
myfile = open(csv_name)
print("File opened in memory")
SQL_STATEMENT = """
        COPY %s FROM STDIN WITH
            CSV
            HEADER
            DELIMITER AS ','
""" % (tbl_name)
cursor.copy_expert(sql=SQL_STATEMENT, file=myfile)
print("File copied to db")

cursor.execute("grant select on table %s to public" % (tbl_name))
conn.commit()

print('table %s import to db complete' % (tbl_name))

In [ ]:
# The following subsections will serve to update the database when additional API requests are made

In [ ]:
last_date = check_date(cursor, tbl_name)[0].strftime('%Y-%m-%d') # checks last date stored in database
print(last_date)

In [ ]:
column_list = [col for col in df.columns]
print(column_list)

In [ ]:
# ---UNFINISHED FUNCTION---
def insert_tab(cursor, tbl_name, col_list):
    insert_query = ("""
        INSERT INTO %s
        

In [ ]:
# ---UNFINISHED FUNCTION---
def app_df2db(cursor, df):
    for i, row in df.iterrows():
        insert_tab(cursor, row)

In [ ]:
cursor.close()
conn.close()